In [22]:
# necessary libraries for prediciton
import utils
import pandas as pd
import numpy as np
import pprint

from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Useful Functions

In [23]:
def pretty_matrix(matrix, row_label, col_label):
    """Pretty print of the given matrix """

    # Restraining labels that are too big
    row_label = [el[:10] + '..' if len(el) > 10 else el
                for el in row_label]
    col_label = [el[:10] + '..' if len(el) > 10 else el
                for el in col_label]

    # Stringfying everything & Joining top label
    s_matrix = [list([" "] + (col_label))] + \
               [[row_label[row_idx]] + \
                [str(e) for e in row] for row_idx, row in enumerate(matrix)]

    # Length of each matrix column
    len_s = [max(map(len, col)) for col in zip(*s_matrix)]

    # Cell formatation
    formatation = '\t'.join('{{:{}}}'.format(x) for x in len_s)

    # Apply cell formation to each matrix element
    pretty_mat = [formatation.format(*row) for row in s_matrix]

    # Print Pretty Matrix
    print('\n'.join(pretty_mat))


def display_confusion_matrix(values):
    '''Display the given array as a confusion matrix'''
    pretty_matrix([values[0:2], values[2:4]],
                  ['Actual NO', 'Actual YES'],
                  ['Predic NO', 'Predic YES'])

In [24]:
def apply_PCA(df, variance_val=0.9, debug=True):
    '''Apply the PCA algorithm to given dataframe,
    using the given variance val to trim the df'''
    # Necessary to normalize all data to use PCA
    scaler=StandardScaler()
    X_scaled=scaler.fit_transform(df)

    # PCA - keep, by default mode, 90% variance
    pca = PCA(variance_val)    
    pca.fit(X_scaled)
    X_pca = pca.transform(X_scaled)

    if debug:
        ex_variance=np.var(X_pca,axis=0)
        ex_variance_ratio = ex_variance/np.sum(ex_variance)
        print(' > Impact in total variance of each generated feature by PCA:')
        print(ex_variance_ratio)

    principal_df = pd.DataFrame(data = X_pca)
    
    return (principal_df, pca)

In [25]:
def auc_scorer(y_true, y_pred):
    '''Scorer of Area Under Curve value'''
    fpr, tpr, _ = metrics.roc_curve(y_true, y_pred)
    return metrics.auc(fpr, tpr)

# Prediction Algorithms

* Decision Tree
* Random Forest
* Gradient Boosting

In [26]:
def create_DT():
    '''Create a new Decision Tree'''
    # Useful DecisionTree tutorial:
    # https://www.datacamp.com/community/tutorials/decision-tree-classification-python
    return DecisionTreeClassifier()

In [27]:
def create_RF():
    '''Create a new Ranfom Forest model'''
    return RandomForestClassifier()

In [28]:
def create_GB():
    '''Create a new Gradient Boosting model'''
    return GradientBoostingClassifier()

# Prediction

* Predictions are done in this notebook.
* It is also useful to compare how serveral algorithms perform against one another.

In [29]:
# Useful Macros
K_FOLD_NUM_SPLITS = 5
SEED = 42
USE_PCA = False

# Pretty printer
pp = pprint.PrettyPrinter(indent=4)

In [30]:
dataset =  utils.read_csv_to_df('dataset/preprocessed_data.csv')
display(dataset.head())

,amount,payments,frequency,balance_mean,balance_max,balance_min,last_ballance,credit_mean,credit_count,credit_max,...,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,loan_to_account_age_days,owner_age_on_loan,salary_to_payments_ratio,status
0,96396,8033,2,12250.000000,20100.0,1100.0,20100.0,5025.000000,4,9900.0,...,9650,3.38,3.67,100,2985,2804,105,0.685425,1.201295,-1
1,165960,4610,1,52083.859459,120512.8,700.0,52208.9,13523.158824,17,36574.0,...,8369,1.79,2.31,117,2854,2618,148,0.241205,1.815401,1
2,127080,2118,1,30060.954167,49590.4,800.0,20272.8,5009.733333,15,19065.0,...,8390,2.28,2.89,132,2080,2122,170,0.922457,3.961284,1
3,105804,2939,1,41297.480000,65898.5,1000.0,34307.3,9254.600000,13,26448.0,...,10045,1.42,1.71,135,6604,6295,185,0.840626,3.417829,1
4,274740,4579,2,57188.211111,122893.1,600.0,41112.9,21255.930769,13,63366.0,...,8288,3.79,4.52,110,1562,1460,204,0.030136,1.810002,1


In [31]:
STATUS_COL = dataset.columns.get_loc("status")

In [32]:
# Setting X and Y
X = dataset.iloc[:, 0:STATUS_COL]
y = dataset.iloc[:, [STATUS_COL]]
display(X.head())

if USE_PCA:
    print(' > Applying PCA to X_train: ')
    X, pca = apply_PCA(X, debug=True)
    display(X.head())

,amount,payments,frequency,balance_mean,balance_max,balance_min,last_ballance,credit_mean,credit_count,credit_max,...,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,loan_to_account_age_days,owner_age_on_loan,salary_to_payments_ratio
0,96396,8033,2,12250.000000,20100.0,1100.0,20100.0,5025.000000,4,9900.0,...,81.8,9650,3.38,3.67,100,2985,2804,105,0.685425,1.201295
1,165960,4610,1,52083.859459,120512.8,700.0,52208.9,13523.158824,17,36574.0,...,73.5,8369,1.79,2.31,117,2854,2618,148,0.241205,1.815401
2,127080,2118,1,30060.954167,49590.4,800.0,20272.8,5009.733333,15,19065.0,...,53.5,8390,2.28,2.89,132,2080,2122,170,0.922457,3.961284
3,105804,2939,1,41297.480000,65898.5,1000.0,34307.3,9254.600000,13,26448.0,...,74.8,10045,1.42,1.71,135,6604,6295,185,0.840626,3.417829
4,274740,4579,2,57188.211111,122893.1,600.0,41112.9,21255.930769,13,63366.0,...,50.5,8288,3.79,4.52,110,1562,1460,204,0.030136,1.810002


# Hyper Parameter Tunning

## Random Search 1st to apporach the best solution, GridSearch to refine it

In [33]:
def getDecisionTreeBest(X, y, debug=True):
    '''Get the Decision Tree Hyper Parameters'''

    # Maximum number of levels in tree
    max_depth = [int(x) for x in range(2, 20, 4)]
    max_depth.append(None)

    # Create the random grid
    grid = {'criterion': ['gini', 'entropy'],
            'splitter': ['best', 'random'],
            'max_features': ['auto', 'sqrt'],
            'max_depth': max_depth,
            'min_samples_split':  [2, 5, 10],
            'min_samples_leaf':  [1, 2, 4]}
    
    if debug:
        pp.pprint(grid)
    
    # Using the grid search for best hyperparameters
    dt = create_DT()
    dt_grid = GridSearchCV(estimator = dt,
                           param_grid = grid,
                           scoring=metrics.make_scorer(auc_scorer,
                                                       greater_is_better=True),
                           cv=K_FOLD_NUM_SPLITS,
                           verbose=2,
                           n_jobs = -1)

    # Fit the grid search model
    dt_grid = dt_grid.fit(X, y)
    
    if debug:
        print('Best Score: ', dt_grid.best_score_)
        print('Best Params: ', dt_grid.best_params_)
        
    # Return score, method & params tuple
    return (dt_grid.best_score_, 'Decision Tree', dt_grid.best_params_)

In [34]:
def getRandomForestBest(X, y, debug=True):
    '''Get the Random Forest Hyper Parameters'''

    # Maximum number of levels in tree
    max_depth = [int(x) for x in range(2, 20, 4)]
    max_depth.append(None)

    # Create the random grid
    grid = {'n_estimators': [int(x) for x in range(2, 14, 2)],
            'max_features': ['auto', 'sqrt'],
            'max_depth': max_depth,
            'criterion': ['gini', 'entropy'],
            'min_samples_split':  [2, 5, 10],
            'min_samples_leaf':  [1, 2, 4],
            'bootstrap': [True, False]}
    
    if debug:
        pp.pprint(grid)
    
    # Using the grid search for best hyperparameters
    rf = create_RF()
    rf_grid = GridSearchCV(estimator = rf,
                           param_grid = grid,
                           scoring=metrics.make_scorer(auc_scorer,
                                                       greater_is_better=True),
                           cv=K_FOLD_NUM_SPLITS,
                           verbose=2,
                           n_jobs = -1)

    # Fit the grid search model
    rf_grid = rf_grid.fit(X, y)
    
    if debug:
        print('Best Score: ', rf_grid.best_score_)
        print('Best Params: ', rf_grid.best_params_)
        
    # Return score, method & params tuple
    return (rf_grid.best_score_, 'Random Forest', rf_grid.best_params_)

In [35]:
def getGradientBoostBest(X, y, debug=True):
    '''Get the Gradient Boost Hyper Parameters'''

    # Create the grid parameters
    grid = {'n_estimators': [int(x) for x in range(2, 14, 2)],
            'learning_rate': [0.1, 0.3, 0.5, 0.7],
            'loss': ['deviance', 'exponential'],
            'criterion': ['friedman_mse', 'mse', 'mae'],
            'min_samples_split':  [2, 5, 10],
            'min_samples_leaf':  [1, 2, 4],
            'random_state': [SEED]}
    
    if debug:
        pp.pprint(grid)
    
    # Using the grid search for best hyperparameters
    gb = create_GB()
    gb_grid = GridSearchCV(estimator = gb,
                           param_grid = grid,
                           scoring=metrics.make_scorer(auc_scorer,
                                                       greater_is_better=True),
                           cv=K_FOLD_NUM_SPLITS,
                           verbose=2,
                           n_jobs = -1)

    # Fit the grid search model
    gb_grid = gb_grid.fit(X, y)
    
    if debug:
        print('Best Score: ', gb_grid.best_score_)
        print('Best Params: ', gb_grid.best_params_)
        
    # Return score, method & params tuple
    return (gb_grid.best_score_, 'Gradient Boosting', gb_grid.best_params_)

In [36]:
# Getting the best algorithm
algorithms = [getDecisionTreeBest(X, y),
              getRandomForestBest(X, y),
              getGradientBoostBest(X, y)]
algorithms.sort(reverse=True, key=lambda el: el[0])

for index, entry in enumerate(algorithms):
    print('%i. %s - %f\n---------' % (index + 1, entry[1], entry[0]))
    
print('Best algorithm: %s' % algorithms[0][1])

{   'criterion': ['gini', 'entropy'],
    'max_depth': [2, 6, 10, 14, 18, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'splitter': ['best', 'random']}
Fitting 5 folds for each of 432 candidates, totalling 2160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:    1.8s


KeyboardInterrupt: 

## Using method with higher score with our data

In [37]:
# Cross validation settings
auc_scores = []
confusion_matrixes = []
cv = KFold(n_splits=K_FOLD_NUM_SPLITS, random_state=SEED, shuffle=False)

# CHANGE THIS LINE TO CHANGE THE USED CLASSIFICATION METHOD
# classifier = DecisionTreeClassifier(criterion='entropy', min_samples_leaf=2, min_samples_split=10, splitter='best')
# classifier = GradientBoostingClassifier(criterion='friedman_mse', learning_rate=0.7, loss='deviance',
#                                         min_samples_leaf=2, min_samples_split=2, n_estimators=8, random_state=SEED)
classifier = RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=None, max_features='sqrt',
                                    min_samples_leaf=1, min_samples_split=2, n_estimators=6)


# Applying Cross validation
for train_index, test_index in cv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Training with this fold
    classifier.fit(X_train, y_train)
    
    # Testing & Measuring accuracy
    y_pred = classifier.predict(X_test)
    
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred)
    auc_scores.append(metrics.auc(fpr, tpr))
    confusion_matrixes.append(metrics.confusion_matrix(y_test, y_pred).ravel())

/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/ipykernel_launcher.py:20: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/Users/edgarcarneiro/Documents/University/feup-ecac/project-competition/env/lib/python3.7/site-packages/ipykernel_launcher.py:20: DataConversionW

In [38]:
# Printing the obtained results
print('Classification Method used:', classifier, '\n')
print('AUC scores:', auc_scores)
print('> Average: ', sum(auc_scores)/len(auc_scores))
for cf in confusion_matrixes:
    display_confusion_matrix(cf)

Classification Method used: RandomForestClassifier(bootstrap=False, class_weight=None, criterion='entropy',
                       max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=6,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False) 

AUC scores: [0.5847953216374269, 0.7407407407407407, 0.6077586206896552, 0.6477987421383649, 0.6666666666666667]
> Average:  0.6495520183745709
          	Predic NO	Predic YES
Actual NO 	2        	7         
Actual YES	3        	54        
          	Predic NO	Predic YES
Actual NO 	6        	6         
Actual YES	1        	53        
          	Predic NO	Predic YES
Actual NO 	2        	6         
Actual YES	2        	56        
          	Predic NO	Predic YES
Actual NO 	4 

### After having our model trained we shall use the model on the data to be sumitted in the kaggle

In [39]:
test_dataset =  utils.read_csv_to_df('dataset/test_dataset.csv')
ids = utils.read_csv_to_df('dataset/ids.csv')
display(test_dataset.head())

,amount,payments,frequency,balance_mean,balance_max,balance_min,last_ballance,credit_mean,credit_count,credit_max,...,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,loan_to_account_age_days,owner_age_on_loan,salary_to_payments_ratio,status
0,93960,1566,1,54520.202247,88246.7,800.0,49548.5,12769.220000,30,36637.5,...,8390,2.28,2.89,132,2080,2122,452,0.389496,5.357599,NaN
1,260640,7240,1,31518.182051,88731.8,-718.6,11565.4,9526.772414,29,41871.0,...,8620,1.10,1.25,100,1089,1117,490,0.688919,1.190608,NaN
2,232560,4845,0,40175.612500,79286.6,200.0,45754.0,15302.486047,43,43605.0,...,10045,1.42,1.71,135,6604,6295,630,0.907412,2.073271,NaN
3,221880,3698,1,44440.912676,74216.8,1000.0,38913.4,11797.642857,21,33280.0,...,8899,3.39,3.97,149,2987,2487,311,0.809056,2.406436,NaN
4,38520,3210,1,20231.313158,31302.0,900.0,18914.3,2736.026923,52,8800.0,...,8388,2.41,2.94,87,1658,1668,597,0.894527,2.613084,NaN


In [40]:
# We now remove the Y column with NaNs
test_dataset = test_dataset.iloc[:, 0:STATUS_COL]
display(test_dataset.head())

,amount,payments,frequency,balance_mean,balance_max,balance_min,last_ballance,credit_mean,credit_count,credit_max,...,ratio of urban inhabitants,average salary,unemploymant rate '95,unemploymant rate '96,no. of enterpreneurs per 1000 inhabitants,no. of commited crimes '95,no. of commited crimes '96,loan_to_account_age_days,owner_age_on_loan,salary_to_payments_ratio
0,93960,1566,1,54520.202247,88246.7,800.0,49548.5,12769.220000,30,36637.5,...,53.5,8390,2.28,2.89,132,2080,2122,452,0.389496,5.357599
1,260640,7240,1,31518.182051,88731.8,-718.6,11565.4,9526.772414,29,41871.0,...,52.4,8620,1.10,1.25,100,1089,1117,490,0.688919,1.190608
2,232560,4845,0,40175.612500,79286.6,200.0,45754.0,15302.486047,43,43605.0,...,74.8,10045,1.42,1.71,135,6604,6295,630,0.907412,2.073271
3,221880,3698,1,44440.912676,74216.8,1000.0,38913.4,11797.642857,21,33280.0,...,55.3,8899,3.39,3.97,149,2987,2487,311,0.809056,2.406436
4,38520,3210,1,20231.313158,31302.0,900.0,18914.3,2736.026923,52,8800.0,...,59.1,8388,2.41,2.94,87,1658,1668,597,0.894527,2.613084


In [41]:
if USE_PCA:
    # Using train PCA and classifying
    scaler=StandardScaler()
    X_test_scaled=scaler.fit_transform(test_dataset)
    predictions_df = pd.DataFrame(data = pca.transform(X_test_scaled))
    display(predictions_df)

    final_df = pd.DataFrame()
    final_df['Predicted'] = classifier.predict(predictions_df)
    final_df['Id'] = ids['loan_id']
    
else:
    predictions_df = test_dataset.copy()
    predictions_df['Predicted'] = classifier.predict(predictions_df)
    final_df = ids.merge(predictions_df, on=['amount', 'average salary ', 'last_ballance'])
    final_df = final_df[['loan_id', 'Predicted']]\
                        .rename(columns={
                            'loan_id': 'Id'
                        })\
                        .drop_duplicates()

display(final_df)

,Id,Predicted
0,5895,1
1,7122,-1
2,6173,1
3,6142,1
4,5358,1
...,...,...
349,4989,-1
350,5221,1
351,6402,1
352,5346,1


In [42]:
# Outputting predictions to .csv
# CHANGE FILE NAME TO PRESERVE DIFFERENT INSTANCES
utils.write_df_to_csv(final_df, 'predictions', 'prediction.csv')